In [ ]:
!pip install -r requirements.txt


In [ ]:
# Importing necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Fine-tuning LLaMA using LoRA

# Loading LLaMA tokenizer and model
llama_model_name = "huggyllama/llama-7b"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name).to(device)

# Loading dataset for LLaMA (using wikitext as an example)
llama_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Tokenizing the dataset
def tokenize_function_llama(examples):
    return llama_tokenizer(examples['text'], return_tensors='pt', truncation=True, padding=True)

llama_dataset = llama_dataset.map(tokenize_function_llama, batched=True)

# LoRA configuration for LLaMA
llama_lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05, 
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Applying LoRA to LLaMA model
llama_peft_model = get_peft_model(llama_model, llama_lora_config)

# Training arguments for LLaMA
llama_training_args = TrainingArguments(
    output_dir="./llama_lora",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=10,
)

# Trainer for LLaMA
llama_trainer = Trainer(
    model=llama_peft_model,
    args=llama_training_args,
    train_dataset=llama_dataset,
)

# Fine-tuning LLaMA
llama_trainer.train()

### Fine-tuning BERT using LoRA

# Loading BERT tokenizer and model
bert_model_name = "bert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=2).to(device)

# Loading dataset for BERT (using IMDB movie reviews as an example)
bert_dataset = load_dataset("imdb")

# Tokenizing the dataset
def tokenize_function_bert(examples):
    return bert_tokenizer(examples['text'], padding="max_length", truncation=True)

bert_encoded_dataset = bert_dataset.map(tokenize_function_bert, batched=True)

# LoRA configuration for BERT
bert_lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=["query", "key"], 
    lora_dropout=0.05, 
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Applying LoRA to BERT model
bert_peft_model = get_peft_model(bert_model, bert_lora_config)

# Training arguments for BERT
bert_training_args = TrainingArguments(
    output_dir="./bert_lora",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=10,
)

# Trainer for BERT
bert_trainer = Trainer(
    model=bert_peft_model,
    args=bert_training_args,
    train_dataset=bert_encoded_dataset["train"],
    eval_dataset=bert_encoded_dataset["test"],
)

# Fine-tuning BERT
bert_trainer.train()
